In [4]:
import os
import datadbhandler as dbhandler
import pandas as pd
import numpy as np

PATH = 'D:/Trading/Historical Data/'
CSVPATH = 'NSE EOD/'
DBPATH = 'db/nsedb.db'
MOD_PATH = 'nse_eod_modifiers/'
AMI_PATH = 'amibroker_files/'

os.chdir(PATH)

db = dbhandler.DataDB(DBPATH)

Opening Bhavcopy database db/nsedb.db...


In [2]:
df = dbhandler.pd.read_csv(db.INDEX_CHANGE_MOD_CSV)

In [3]:
df.head()

,Index,Date,Symbol,ChangeType
0,Nifty 50,19960918,ABB,I
1,Nifty 50,19960918,ASIANPAINT,I
2,Nifty 50,19960918,EIHOTEL,I
3,Nifty 50,19960918,GLAXO,I
4,Nifty 50,19960918,M&M,I


In [4]:
df.iloc[0][1]

19960918

In [15]:
db.fetch_records('tblModDump', [df.iloc[0][2]], start_date=df.iloc[0][1], end_date=df.iloc[0][1])

tblModDump
['ABB']
19960918
19960918
19960918 19960918


,Symbol,Date,Open,High,Low,Close,Volume,AdjustedOpen,AdjustedHigh,AdjustedLow,AdjustedClose
0,ABB,19960918,810.0,816.0,809.25,815.25,1600,None,None,None,None


In [14]:
db.fetch_records('tblModDump', [df.iloc[0][2]], start_date=19960918, end_date=19960918)

tblModDump
['ABB']
19960918
19960918
19960918 19960918


,Symbol,Date,Open,High,Low,Close,Volume,AdjustedOpen,AdjustedHigh,AdjustedLow,AdjustedClose
0,ABB,19960918,810.0,816.0,809.25,815.25,1600,None,None,None,None


In [2]:
db.truncate_table('tblDumpReplace', msg=True)

Truncated table tblDumpReplace


In [3]:
db.truncate_table('tblMultipliers', msg=True)

Truncated table tblMultipliers


In [4]:
db.truncate_table('tblSkipped', msg=True)

Truncated table tblSkipped


In [5]:
db.truncate_table('tblSymbolRange', msg=True)

Truncated table tblSymbolRange


In [6]:
db.truncate_table('tblHistIndex', msg=True)

Truncated table tblHistIndex


In [7]:
db.truncate_table('tblDuplicates', msg=True)

Truncated table tblDuplicates


In [11]:
for i in range(1995, 2020):
    db.truncate_table('tblModDump{}'.format(i), msg=True)

Truncated table tblModDump1995
Truncated table tblModDump1996
Truncated table tblModDump1997
Truncated table tblModDump1998
Truncated table tblModDump1999
Truncated table tblModDump2000
Truncated table tblModDump2001
Truncated table tblModDump2002
Truncated table tblModDump2003
Truncated table tblModDump2004
Truncated table tblModDump2005
Truncated table tblModDump2006
Truncated table tblModDump2007
Truncated table tblModDump2008
Truncated table tblModDump2009
Truncated table tblModDump2010
Truncated table tblModDump2011
Truncated table tblModDump2012
Truncated table tblModDump2013
Truncated table tblModDump2014
Truncated table tblModDump2015
Truncated table tblModDump2016
Truncated table tblModDump2017
Truncated table tblModDump2018


OperationalError: no such table: tblModDump2019

In [2]:
import pandas as pd

In [7]:
df = dbhandler.pd.read_csv(db.SYMBOL_MAPPING_FILE)

In [80]:
df2 = dbhandler.pd.read_csv(db.INDEX_CHANGE_MOD_CSV)
df2.Symbol.value_counts()

ABBOTINDIA    13
TCS           12
BAYERCROP      8
BAYER          4
Name: Symbol, dtype: int64

In [81]:
import numpy as np
manual = dbhandler.pd.read_csv(db.INDEX_CHANGE_MANUAL_CSV, encoding='ansi')

for symbol in ['BAYERCROP', 'ABBOTINDIA']: #manual.Symbol.unique():
    symbol_dates = manual[manual.Symbol == symbol]['Date'].unique()
    #print(1, symbol, symbol_dates)
    #print(2, manual[manual.Symbol == symbol].CorrectSymbol)
    correct_symbol = manual[manual.Symbol == symbol].CorrectSymbol.iloc[0]
    #print(3, symbol, correct_symbol, symbol_dates)
    for date in symbol_dates:
        #print(4, symbol, date, correct_symbol)
        #print(5, np.where((df2.Symbol == symbol) & (df2.Date == date), correct_symbol, df2.Symbol))
        #print(5, np.where(df2.Date == date, correct_symbol, df2.Symbol))
        #print(0, symbol, date, correct_symbol)
        #print(1, df2[df2.Symbol == symbol])
        #print(2, df2[df2.Date == date])
        #print(3, df2[(df2.Symbol == symbol) & (df2.Date == date)])
        df2['Symbol'] = np.where((df2.Symbol == symbol) & (df2.Date == int(date)), correct_symbol, df2.Symbol)
df2.to_csv(db.INDEX_CHANGE_MOD_CSV)

In [82]:
df2 = dbhandler.pd.read_csv(db.INDEX_CHANGE_MOD_CSV)
df2.Symbol.value_counts()

TCS           12
BAYER         10
ABBOTINDIA     7
PHARMACIA      6
BAYERCROP      2
Name: Symbol, dtype: int64

In [67]:
df

,Index,Date,Symbol,ChangeType
8,Nifty 500,19980801,BAYERCROP,I
21,Nifty MNC,19980801,BAYERCROP,I


In [2]:
import pandas as pd

In [5]:
symbols_range = pd.read_sql_query('''SELECT Symbol, MIN(Date) MinDate, MAX(Date) MaxDate 
                                        FROM tblModDump{} WHERE Date >= {} 
                                        GROUP BY Symbol'''.format(2020, 20200805), db.conn)

In [7]:
symbols_range.index = symbols_range.Symbol
symbols_range

,Symbol,MinDate,MaxDate
Symbol,,,
20MICRONS,20MICRONS,20200805,20200806
21STCENMGM,21STCENMGM,20200805,20200806
3IINFOTECH,3IINFOTECH,20200805,20200806
3MINDIA,3MINDIA,20200805,20200806
3PLAND,3PLAND,20200805,20200806
...,...,...,...
ZODJRDMKJ,ZODJRDMKJ,20200805,20200806
ZOTA,ZOTA,20200805,20200806
ZUARI,ZUARI,20200805,20200806


In [1]:
start_date = '20200805'
qry = '''SELECT Symbol, Min(Date) MinDate, Max(Date) MaxDate, Source, TableSource, StartDate, EndDate
                FROM (SELECT t1.Symbol, t1.Date, 'Dump' Source, t2.TableSource, t2.StartDate, t2.EndDate
		FROM tblDump{year} t1
		LEFT JOIN tblSymbolRange t2
		ON t1.Symbol = t2.Symbol
		AND TableSource <> 'ModDump'
		WHERE t1.Date >= {start_date}
		UNION
	SELECT t1.Symbol, t1.Date, 'ModDump' Source, t2.TableSource, t2.StartDate, t2.EndDate
		FROM tblModDump{year} t1
		LEFT JOIN tblSymbolRange t2
		ON t1.Symbol = t2.Symbol
		AND TableSource <> 'Dump'
		WHERE t1.Date >= {start_date}
		UNION
        SELECT t1.Symbol, t1.Date, 'Dump' Source, t2.TableSource, t2.StartDate, t2.EndDate
		FROM tblDump{year} t1
		LEFT JOIN tblSymbolRange t2
		ON t1.Symbol = t2.Symbol
		WHERE t1.Date >= {start_date}
		UNION
	SELECT t1.Symbol, t1.Date, 'ModDump' Source, t2.TableSource, t2.StartDate, t2.EndDate
		FROM tblModDump{year} t1
		LEFT JOIN tblSymbolRange t2
		ON t1.Symbol = t2.Symbol
		WHERE t1.Date >= {start_date}) t1
        WHERE Source = TableSource 
        OR TableSource IS NULL
        GROUP BY Symbol, Source, TableSource'''.format(start_date=start_date, year=start_date[0:4])

In [15]:
join = pd.read_sql_query(qry, db.conn)

In [16]:
join

,Symbol,MinDate,MaxDate,Source,TableSource,StartDate,EndDate
0,20MICRONS,20200805,20200807,Dump,Dump,20200805,20200807
1,20MICRONS,20200805,20200807,ModDump,ModDump,20200805,20200807
2,21STCENMGM,20200805,20200807,Dump,Dump,20200805,20200807
3,21STCENMGM,20200805,20200807,ModDump,ModDump,19951024,20200807
4,3IINFOTECH,20200805,20200807,Dump,Dump,20050422,20200807
...,...,...,...,...,...,...,...
3459,ZUARI,20200805,20200807,ModDump,ModDump,20121127,20200807
3460,ZUARIGLOB,20200805,20200807,Dump,Dump,20120925,20200807
3461,ZUARIGLOB,20200805,20200807,ModDump,ModDump,19950102,20200807
3462,ZYDUSWELL,20200805,20200807,Dump,Dump,20091113,20200807
